___

<a> <img src='DataWorkflow_LowRes.png' width="600"/></a>
___
# Data Workflow Crash Course

Please note, this is not meant to be a comprehensive overview of Python or programming or recomended data workflows. It is simply to give some examples of what your process may resemble.

**This notebook is just a potential reference, any and all of the example can be altered or a different approach taken altogether.**
____

<div align="center"><b>
    <h3>Other Notebooks</h3>
    <h4> <a href="0%20-%20Data%20Workflow%20Overview.ipynb">0 - Data Workflow Overview</a></h4>
    <h4> <a href="1 - Data Acquisition.ipynb">1 - Data Aquisition</a></h4>
    <h4> <a href="2 - Processing.ipynb">2 - Processing</a></h4>
    <h4> <a href="3 - Analysis.ipynb">3 - Analysis</a></h4>
    <h4> <a href="4 - Permanent Storage.ipynb">4 - Permanent Storage</a></h4>
</b><div align="center">

## Processing
This is the stage where the data will be combined and turned into usable formats for analysis. This stage is often combined with the analyis stage. The main goal is to make datasets ready for exploratory data analysis, comparison, and statistical analyis. This is often referred to as data cleanup and may take place in Excel, Python, or other programs.

### Import Libraries for Use

In [120]:
import pandas as pd
import datetime as dt
import numpy as np

### Check Data
When checking data it's useful to look at:
* Data types
* Missing data
* Matching dates or labels between datasets
* Variable labels
* How will it be used

In [121]:
# Import dataframes
df_weather = pd.read_excel('Example_HourlyWeatherData.xlsx')
df_lab = pd.read_excel('Example_MultivariateExperiments.xlsx')
df_timeseries = pd.read_excel('Example_TimeSeriesProductivity.xlsx')

In [122]:
# An example of some useful functions to check data
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        8760 non-null   datetime64[ns]
 1   Light       8760 non-null   float64       
 2   PPF         8760 non-null   int64         
 3   Air_Temp    8760 non-null   float64       
 4   Rel_Hum     8760 non-null   float64       
 5   Water_Temp  8760 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 410.8 KB


In [123]:
df_weather.head()

,Date,Light,PPF,Air_Temp,Rel_Hum,Water_Temp
0,2021-01-01 00:00:00,559.485333,0,-2.9,79.798762,12.74412
1,2021-01-01 01:00:00,559.485333,0,-3.2,80.364525,12.63696
2,2021-01-01 02:00:00,559.485333,0,-3.2,79.753412,12.63696
3,2021-01-01 03:00:00,559.485333,0,-3.3,80.349778,12.60124
4,2021-01-01 04:00:00,559.485333,0,-3.4,80.335015,12.56552


In [124]:
df_weather.tail()

,Date,Light,PPF,Air_Temp,Rel_Hum,Water_Temp
8755,2021-12-31 19:00:00,558.588664,0,-2.1,77.532570,13.02988
8756,2021-12-31 20:00:00,558.588664,0,-2.3,78.090880,12.95844
8757,2021-12-31 21:00:00,558.588664,0,-2.4,78.074717,12.92272
8758,2021-12-31 22:00:00,558.588664,0,-2.6,78.638823,12.85128
8759,2021-12-31 23:00:00,558.588664,0,-2.8,79.208449,12.77984


In [125]:
df_weather.shape

(8760, 6)

In [126]:
df_weather.dtypes

Date          datetime64[ns]
Light                float64
PPF                    int64
Air_Temp             float64
Rel_Hum              float64
Water_Temp           float64
dtype: object

In [127]:
df_weather.describe()

,Light,PPF,Air_Temp,Rel_Hum,Water_Temp
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,732.392638,494.420548,11.643231,52.881731,17.938962
std,120.351823,650.885631,10.186807,17.831317,3.638728
min,555.130035,0.000000,-4.000000,18.500581,12.351200
25%,615.828655,0.000000,2.600000,38.524728,14.708720
50%,734.177581,0.000000,10.800000,53.543668,17.637760
75%,849.549548,971.250000,20.025000,67.999906,20.932930
max,905.515841,2049.000000,32.800000,81.599258,25.496160


### Combining Datasets
In the time-series example the weather data and biomass productivity data need to be combined for analysis. With these it is also important to determine how it will be analyzed. Some potential methods are:
* Average weather values for the date ranges of each producitivity measurement.
* Average daily values for weather and fill in the information for productivity for each time period. This could be useful if planning on time-series decomposition later.

There are surely many more methods but I'll focus on these two as examples.

#### Averaging Weather Data per Productivity Time Range
The goal is essentially to group data by dates when biomass measurements were made.

First we need to fill PAR values of 0 with NaN so they are not included in averages. Because we already have light duration as an input we want the PAR averages to reflect average intensity during the day.

You could also integrate PAR values and get a total of PAR during the day in (µmol m<sup>-2</sup>)

In [128]:
# Copy dataframe so original data is not changed
df_weather_proc = df_weather

# Select column
cols = ["PPF"]

# Replace 0 with NaN
df_weather_proc[cols] = df_weather_proc[cols].replace({'0':np.nan, 0:np.nan})

# Groupby daily averages based on hourly data.
df_weather_proc = df_weather_proc.set_index('Date').groupby(pd.Grouper(freq='D')).mean()

In [129]:
# Create datetime index
df_timeseries_proc = df_timeseries.set_index('Date')

In [130]:
# Select first date of weather data for comparison
start_date = '2021-01-01' #  00:00:00
end_date = start_date

# Create dataframe for combining values
df_grouped_timeseries = pd.DataFrame(columns = ['Date', 'Light', 'PPF', 'Air_Temp', 'Water_Temp', 'Productivity'])

# Loop through selecting and averaging date ranges
for ind in df_timeseries_proc.index:
    start_date = end_date
    end_date = str(ind)
    df = df_weather_proc.loc[start_date:end_date]
    df = df.mean(axis='index')
    df_grouped_timeseries = df_grouped_timeseries.append({
        'Date' : end_date, 'Light' : df['Light'],
        'PPF' : df['PPF'], 'Air_Temp' : df['Air_Temp'],
        'Water_Temp' : df['Water_Temp'], 'Productivity' : df_timeseries_proc['Productivity'][ind]}, 
                ignore_index = True)

# Create datetime index
df_grouped_timeseries = df_grouped_timeseries.set_index('Date')
df_grouped_timeseries.index = pd.to_datetime(df_grouped_timeseries.index)

# Export data to Excel for Analysis stage
df_grouped_timeseries.to_excel("Exports/Grouped Timeseries.xlsx")

#### Fill in Values for Productivity Across Ranges
Instead of grouping this approach will use measured productivity to approximate productivity for each day before it.

In [131]:
# Copy dataframe so original data is not changed
df_weather_proc = df_weather

# Select column
cols = ["PPF"]

# Replace 0 with NaN
df_weather_proc[cols] = df_weather_proc[cols].replace({'0':np.nan, 0:np.nan})

# Groupby daily averages based on hourly data.
df_weather_proc = df_weather_proc.set_index('Date').groupby(pd.Grouper(freq='D')).mean()

In [132]:
# Create datetime index
df_timeseries_proc = df_timeseries.set_index('Date')

In [133]:
# Merge weather and productivity dataframes based on matching dateime index
df_daily_timeseries = pd.merge(df_weather_proc, df_timeseries_proc, how='left', left_index=True, right_index=True)

# First use backfill to fill ranges based on end value the ffill for any missed values
df_daily_timeseries = df_daily_timeseries.fillna(method='backfill')
df_daily_timeseries = df_daily_timeseries.fillna(method='ffill')

# Export data to Excel for Analysis stage
df_daily_timeseries.to_excel("Exports/Daily Timeseries.xlsx")

[<img src="https://imgs.xkcd.com/comics/flawed_data.png">](https://xkcd.com/2494/)